In [24]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

In [25]:
merge_left = pd.read_csv('../data/merged_lead_student.csv')

In [26]:
merge_left.columns

Index(['Unnamed: 0', 'email', 'nome', 'lead_scoring_-_perfil', 'url_pública',
       'estágio_no_funil', 'total_de_conversões', 'lead_scoring_-_interesse',
       'status_para_comunicação_por_email', 'data_da_primeira_conversão',
       'eventos_(últimos_100)', 'origem_da_última_conversão',
       'data_da_última_conversão', 'origem_da_primeira_conversão', 'empresa',
       'tags', 'data_da_última_oportunidade', 'estado',
       'valor_total_da_oportunidade_no_crm_(última_atualização)',
       'qualificação_da_oportunidade_no_crm_(última_atualização)',
       'etapa_do_funil_de_vendas_no_crm_(última_atualização)',
       'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
       'origem_da_oportunidade_no_crm_(última_atualização)',
       'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?',
       'desejo_receber_o_sustentare_news', 'como_você_conheceu_a_sustentare:',
       'universidade', 'qual_o_curso_de_interesse?', 'cargo_final',
       'area_atuacao', 'inter

In [27]:
merge_left = merge_left.drop(columns='Unnamed: 0')

### Change dtype for columns that are date

In [28]:
def convert_columns_to_datetime(df):
    """
    Convert specified columns in a DataFrame to datetime format.

    Parameters:
    - DataFrame

    Returns:
    - The DataFrame with specified columns converted to datetime.
    """

    date_columns = ['data_da_primeira_conversão', 'data_da_última_conversão', 'data_da_última_oportunidade', 'data_nascimento', 'min_dt_log']
    
    df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce'))
    for i in date_columns:
        df[i] = pd.to_datetime(df[i], utc=True).dt.tz_convert(None)
    return df

merge_left = convert_columns_to_datetime(merge_left)
merge_left.dtypes


C:\Users\anafl\AppData\Local\Temp\ipykernel_47664\2429628830.py:14: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce'))
C:\Users\anafl\AppData\Local\Temp\ipykernel_47664\2429628830.py:14: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise a warning unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df[date_columns] = df[date_columns].apply(lambda col: pd.to_datetime(col, errors='coerce'))
C:\Users\anafl\AppData\Local\Temp\

email                                                                        object
nome                                                                         object
lead_scoring_-_perfil                                                        object
url_pública                                                                  object
estágio_no_funil                                                             object
total_de_conversões                                                           int64
lead_scoring_-_interesse                                                      int64
status_para_comunicação_por_email                                              bool
data_da_primeira_conversão                                           datetime64[ns]
eventos_(últimos_100)                                                        object
origem_da_última_conversão                                                   object
data_da_última_conversão                                             datetim

In [29]:
def calculate_months_since_conversion(df):
    """
    Calculates the number of months since the first conversion based on the 'aluno' column and additional date columns.
    Drops rows where 'aluno' = 1 and min_dt_log is null.

    Parameters:
    - The input DataFrame containing the necessary columns.
    
    Returns:
    - The DataFrame with additional columns representing the number of months since the first conversion,
          last conversion, and last opportunity. Null values are assigned to these columns if the corresponding dates are null.
    """
    # Drop rows based on conditions
    df = df.drop(df[(df['aluno'] == 1) & df['min_dt_log'].isnull()].index)

    # Calculate months since conversion
    df['months_since_first_conversion'] = np.where(
        df['aluno'] == 1,
        (df['min_dt_log'] - df['data_da_primeira_conversão']).dt.days / 30,
        (datetime.now() - df['data_da_primeira_conversão']).dt.days / 30
    )

    # Calculate months since last conversion
    df['months_since_last_conversion'] = np.where(
        df['data_da_última_conversão'].notnull(),
        (df['data_da_última_conversão'] - df['data_da_primeira_conversão']).dt.days / 30,
        np.nan
    )

    # Calculate months since last opportunity
    df['months_since_last_opportunity'] = np.where(
        df['data_da_última_oportunidade'].notnull(),
        (df['data_da_última_oportunidade'] - df['data_da_primeira_conversão']).dt.days / 30,
        np.nan
    )

    #df = df.drop(columns=['data_da_primeira_conversão', 'min_dt_log', 'data_da_última_conversão', 'data_da_última_oportunidade'])

    return df

perfil_df = calculate_months_since_conversion(merge_left)


## Creating a Function for the Age

In [30]:
def create_column_with_calculate_age(df):
    """
    Calculates age based on the birthdate column and adds a new column to the DataFrame.

    Parameters:
    - DataFrame.

    Returns:
    - The DataFrame with the new age column added.
    """
    df['age'] = pd.to_datetime(df['data_nascimento'], errors='coerce').dt.year
    current_year = pd.to_datetime('now').year
    df['age'] = current_year - df['age']

    # Set age to NaN for values less than 19 or greater than 70
    df.loc[(df['age'] < 19) | (df['age'] > 70), 'age'] = pd.NA
    
    #df = df.drop(columns='data_nascimento')

    return df

perfil_df = create_column_with_calculate_age(perfil_df)


In [31]:
perfil_df['eventos_(últimos_100)'].value_counts()

eventos_(últimos_100)
Banco Easy2Mail Nome+Email                                                                                                                                                                                                             5057
Transferência mail2easy RD-Interessados / Banco Easy2Mail Nome+Email                                                                                                                                                                    820
e-book-desafios-equipes                                                                                                                                                                                                                 649
inscricao-festival-rh-agil-3-edicao                                                                                                                                                                                                     608
ebook-o-imprescindivel-processo-de

# Dealing with evento column

Based on the analysis on the exploration file a few steps were made to obtain a clear data set

In [32]:
import re

def add_keyword_columns(dataframe):
    """
    Add new columns for each specific keyword and populate them with the counts of occurrences.
    Also, add a new column that represents the sum of values across all keyword columns.

    Parameters:
    - DataFrame 

    Returns:
    - The modified DataFrame with added columns.
    """

    keywords = ['live', 'newsletter', 'master-class', 'whatsapp', 'webinar', 'gratuito', 'gratuita' , 'festival', 
                'linkedin', 'ebook', 'exame-de-bolsas', 'e-book', 'bolsa-de-estudos']

    # Create a regex pattern to match variations of the keywords
    pattern = re.compile(r'\b(?:' + '|'.join(keywords) + r')\b', flags=re.IGNORECASE)

    # Convert the entire column to string
    dataframe['eventos_(últimos_100)'] = dataframe['eventos_(últimos_100)'].astype(str)

    # Iterate through each keyword and add a new column
    for i in keywords:
        # Create a new column for each keyword
        dataframe[i] = 0

        # Iterate through each row and count occurrences
        for idx, row in dataframe.iterrows():
            events = row['eventos_(últimos_100)'].split('/')

            # Check if the keyword appears in the events using regex
            for event in events:
                if pattern.search(event):
                    matched_keyword = pattern.search(event).group().lower()
                    if matched_keyword == i.lower():
                        # Increment the count in the corresponding column
                        dataframe.at[idx, i] += 1

    # Add a new column for the sum of keyword columns
    dataframe['total_interacoes'] = dataframe[keywords].sum(axis=1)

    return dataframe

new_perfil_df = add_keyword_columns(perfil_df)
new_perfil_df.sample()


,email,nome,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),...,webinar,gratuito,gratuita,festival,linkedin,ebook,exame-de-bolsas,e-book,bolsa-de-estudos,total_interacoes
34267,adriseguranca@yahoo.com.br,Adriana Fugazza,d,http://app.rdstation.com.br/leads/public/21c51...,Lead,1,0,True,2015-07-14 19:40:40,ex-alunos-sustentare,...,0,0,0,0,0,0,0,0,0,0


In [33]:
def add_keyword_columns(dataframe):
    """
    Add new columns for each specific keyword and populate them with the counts of occurrences.
    Also, add a new column that represents the sum of values across all keyword columns.
    Additionally, add a new column 'total_eventos' that counts the number of values separated by "|".

    Parameters:
    - The DataFrame containing the data.

    Returns:
    - The modified DataFrame with added columns.
    """

    keywords = ['live', 'newsletter', 'master-class', 'whatsapp', 'webinar', 'gratuito', 'gratuita' , 'festival', 
                'linkedin', 'ebook', 'exame-de-bolsas', 'e-book', 'bolsa-de-estudos']

    # Create a regex pattern to match variations of the keywords
    pattern = re.compile(r'\b(?:' + '|'.join(keywords) + r')\b', flags=re.IGNORECASE)

    # Iterate through each keyword and add a new column
    for i in keywords:
        # Create a new column for each keyword
        dataframe[i] = 0

        # Iterate through each row and count occurrences
        for idx, row in dataframe.iterrows():
            events = row['eventos_(últimos_100)'].split('/')

            # Check if the keyword appears in the events using regex
            for event in events:
                if pattern.search(event):
                    matched_keyword = pattern.search(event).group().lower()
                    if matched_keyword == i.lower():
                        # Increment the count in the corresponding column
                        dataframe.at[idx, i] += 1

    # Add a new column for the sum of keyword columns
    dataframe['total_interacoes'] = dataframe[keywords].sum(axis=1)

    # Add a new column for the count of values in 'eventos_(últimos_100)'
    dataframe['total_eventos'] = dataframe['eventos_(últimos_100)'].apply(lambda x: x.count('/') + 1)

    return dataframe

new_perfil_df = add_keyword_columns(perfil_df)
new_perfil_df.sample()

,email,nome,lead_scoring_-_perfil,url_pública,estágio_no_funil,total_de_conversões,lead_scoring_-_interesse,status_para_comunicação_por_email,data_da_primeira_conversão,eventos_(últimos_100),...,gratuito,gratuita,festival,linkedin,ebook,exame-de-bolsas,e-book,bolsa-de-estudos,total_interacoes,total_eventos
11969,apcastroalmeida@gmail.com,Ana paula Almeida,d,http://app.rdstation.com.br/leads/public/fe962...,Lead,1,10,True,2021-08-09 17:57:31,inscricao-festival-rh-agil,...,0,0,1,0,0,0,0,0,1,1


In [34]:
new_perfil_df['total_eventos'].value_counts()

total_eventos
1      21614
2       6854
3       2958
4       1422
5        855
       ...  
114        1
193        1
84         1
100        1
104        1
Name: count, Length: 89, dtype: int64

In [35]:
new_perfil_df.columns

Index(['email', 'nome', 'lead_scoring_-_perfil', 'url_pública',
       'estágio_no_funil', 'total_de_conversões', 'lead_scoring_-_interesse',
       'status_para_comunicação_por_email', 'data_da_primeira_conversão',
       'eventos_(últimos_100)', 'origem_da_última_conversão',
       'data_da_última_conversão', 'origem_da_primeira_conversão', 'empresa',
       'tags', 'data_da_última_oportunidade', 'estado',
       'valor_total_da_oportunidade_no_crm_(última_atualização)',
       'qualificação_da_oportunidade_no_crm_(última_atualização)',
       'etapa_do_funil_de_vendas_no_crm_(última_atualização)',
       'nome_do_responsável_pela_oportunidade_no_crm_(última_atualização)',
       'origem_da_oportunidade_no_crm_(última_atualização)',
       'possui_interesse_em_realizar_uma_pós-graduação_ou_mba?',
       'desejo_receber_o_sustentare_news', 'como_você_conheceu_a_sustentare:',
       'universidade', 'qual_o_curso_de_interesse?', 'cargo_final',
       'area_atuacao', 'interesse_final', '

Some columns represents the same event

In [36]:
new_perfil_df['gratuito'] = new_perfil_df['gratuito'] + new_perfil_df['gratuita']

# Drop the 'gratuita' column 
new_perfil_df = new_perfil_df.drop(columns=['gratuita'])


In [37]:
new_perfil_df['ebook'] = new_perfil_df['ebook'] + new_perfil_df['e-book']

# Drop the 'e-book' column 
new_perfil_df = new_perfil_df.drop(columns=['e-book'])

In [38]:
new_perfil_df.isna().sum()

email                                                                    0
nome                                                                   681
lead_scoring_-_perfil                                                    0
url_pública                                                              0
estágio_no_funil                                                         0
total_de_conversões                                                      0
lead_scoring_-_interesse                                                 0
status_para_comunicação_por_email                                        0
data_da_primeira_conversão                                             333
eventos_(últimos_100)                                                    0
origem_da_última_conversão                                             333
data_da_última_conversão                                               333
origem_da_primeira_conversão                                           333
empresa                  

In [39]:
new_perfil_df.to_csv('../data/clear_perfil_visitor_and_students.csv', index=False)